<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/데이콘/기사추천시스템/open/'

In [4]:
article_info = pd.read_csv(data_path + 'article_info.csv')

In [5]:
article_info #기사 정보(userid = 기사 작성자)

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [6]:
article_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   articleID    3008 non-null   object
 1   Title        3008 non-null   object
 2   Content      3008 non-null   object
 3   Format       3008 non-null   object
 4   Language     3008 non-null   object
 5   userID       3008 non-null   object
 6   userCountry  659 non-null    object
 7   userRegion   657 non-null    object
dtypes: object(8)
memory usage: 188.1+ KB


In [7]:
article_info

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [8]:
view_log = pd.read_csv(data_path + 'view_log.csv')

In [9]:
view_log #사용자-기사 데이터

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [10]:
view_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       42717 non-null  object
 1   articleID    42717 non-null  object
 2   userRegion   42717 non-null  object
 3   userCountry  42717 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


# TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [12]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) #stop_words='english'는 영어 불용어를 제거하고, max_features=5000는 최대 5000개의 단어 특성을 사용한다는 의미입니다.
tfidf_matrix = tfidf_vectorizer.fit_transform(article_info['Title'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [ ]:
def get_personalized_recommendations(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 상위 5개 기사 추천
    recommended_article_indices = ranked_indices[:5]

    return articles_df.iloc[recommended_article_indices]



In [ ]:
get_personalized_recommendations('USER_1420')['articleID'].values

array(['ARTICLE_1351', 'ARTICLE_1891', 'ARTICLE_1788', 'ARTICLE_0445',
       'ARTICLE_2409'], dtype=object)

In [ ]:
recommendations = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations.append([user, get_personalized_recommendations(user).articleID.values])

In [ ]:
recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

In [ ]:
recommendations = recommendations.explode('articleID').reset_index(drop=True)


In [ ]:
submission = recommendations.to_csv(data_path + 'submission_v2.csv')

In [ ]:
submission

In [ ]:
data_path

'/content/drive/MyDrive/데이콘/기사추천시스템/open/'

# word2vec

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

## 1) no remove stopword

In [16]:
# 기사 제목을 단어 리스트로 변환
sentences = [title.split() for title in article_info['Title']]

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 각 문서의 벡터를 평균하여 문서 벡터 생성
def document_vector(doc):
    # 해당 문서의 단어 벡터를 가져와서 평균을 계산
    return np.mean([w2v_model.wv[word] for word in doc if word in w2v_model.wv], axis=0)

# 모든 기사의 벡터를 계산
article_vectors = np.array([document_vector(title.split()) for title in article_info['Title']])


In [18]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(article_vectors)

def get_personalized_recommendations_w2v(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 상위 5개 기사 추천
    recommended_article_indices = ranked_indices[:5]

    return articles_df.iloc[recommended_article_indices]

# 예시 사용법
user_id = 'USER_0000'
recommended_articles = get_personalized_recommendations_w2v(user_id)
print(recommended_articles)


         articleID                                              Title  \
826   ARTICLE_0826  How to forecast demand with Google BigQuery, p...   
1861  ARTICLE_1861  How Technology Hijacks People's Minds - from a...   
150   ARTICLE_0150  How a Japanese cucumber farmer is using deep l...   
1238  ARTICLE_1238  Google Cloud Dataproc - the fast, easy and saf...   
844   ARTICLE_0844  Brazilian Government and Ericsson to innovate ...   

                                                Content Format Language  \
826   Demand forecasting is something that every bus...   HTML       en   
1861  How Technology Hijacks People's Minds - from a...   HTML       en   
150   It's not hyperbole to say that use cases for m...   HTML       en   
1238  Posted by James Malone, Product Manager Please...   HTML       en   
844   Public-private partnership supports creation o...   HTML       en   

         userID userCountry userRegion  
826   USER_0222         NaN        NaN  
1861  USER_0416         NaN 

In [19]:
recommendations_word2vec = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations_word2vec.append([user, get_personalized_recommendations_w2v(user).articleID.values])

In [20]:
recommendations_word2vec = pd.DataFrame(recommendations_word2vec, columns=['userID', 'articleID'])

In [21]:
recommendations_word2vec

,userID,articleID
0,USER_0000,"[ARTICLE_0826, ARTICLE_1861, ARTICLE_0150, ART..."
1,USER_0001,"[ARTICLE_1437, ARTICLE_1861, ARTICLE_0906, ART..."
2,USER_0002,"[ARTICLE_2990, ARTICLE_2204, ARTICLE_0906, ART..."
3,USER_0003,"[ARTICLE_2994, ARTICLE_0782, ARTICLE_1104, ART..."
4,USER_0004,"[ARTICLE_1861, ARTICLE_2198, ARTICLE_0763, ART..."
...,...,...
1410,USER_1416,"[ARTICLE_0399, ARTICLE_2703, ARTICLE_0813, ART..."
1411,USER_1417,"[ARTICLE_1486, ARTICLE_2198, ARTICLE_2868, ART..."
1412,USER_1418,"[ARTICLE_0535, ARTICLE_1997, ARTICLE_0774, ART..."
1413,USER_1419,"[ARTICLE_0439, ARTICLE_0867, ARTICLE_0693, ART..."


In [22]:
recommendations_word2vec2 = recommendations_word2vec.explode('articleID').reset_index(drop=True)


In [ ]:
recommendations_word2vec2.to_csv(data_path + 'submission_w2v_v2.csv')

In [23]:
recommendations_word2vec2

,userID,articleID
0,USER_0000,ARTICLE_0826
1,USER_0000,ARTICLE_1861
2,USER_0000,ARTICLE_0150
3,USER_0000,ARTICLE_1238
4,USER_0000,ARTICLE_0844
...,...,...
7070,USER_1420,ARTICLE_1437
7071,USER_1420,ARTICLE_1486
7072,USER_1420,ARTICLE_2033
7073,USER_1420,ARTICLE_1913


## 2) remove stopword

In [29]:
# nltk 불용어 다운로드
import re
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 기사 제목을 전처리하고 단어 리스트로 변환
def preprocess(text):
    # 소문자 변환, 특수문자 제거, 불용어 제거
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return words

sentences = [preprocess(title) for title in article_info['Title']]

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 각 문서의 벡터를 평균하여 문서 벡터 생성
def document_vector(doc):
    # 해당 문서의 단어 벡터를 가져와서 평균을 계산
    return np.mean([w2v_model.wv[word] for word in doc if word in w2v_model.wv], axis=0)

# 모든 기사의 벡터를 계산
article_vectors = np.array([document_vector(preprocess(title)) for title in article_info['Title']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(article_vectors)

def get_personalized_recommendations(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 사용자가 이미 읽은 기사 제외
    recommended_article_indices = [idx for idx in ranked_indices if idx not in article_indices][:5]

    return articles_df.iloc[recommended_article_indices]

# 예시 사용법
user_id = 'USER_0000'
recommended_articles = get_personalized_recommendations(user_id)
print(recommended_articles)


         articleID                                              Title  \
2572  ARTICLE_2572  Google Save, serviço para salvar coisas da web...   
150   ARTICLE_0150  How a Japanese cucumber farmer is using deep l...   
2461  ARTICLE_2461                   The Blockchain is the new Google   
1360  ARTICLE_1360  Google Cloud Machine Learning family grows wit...   
252   ARTICLE_0252  Google lança teclado virtual para iPhone com r...   

                                                Content Format Language  \
2572  Restrito aos Estados Unidos desde seu lançamen...   HTML       pt   
150   It's not hyperbole to say that use cases for m...   HTML       en   
2461  The following is an excerpt from The Business ...   HTML       en   
1360  Google Cloud Machine Learning is one of our fa...   HTML       en   
252   O Google anunciou hoje (12) o lançamento do Gb...   HTML       pt   

         userID userCountry userRegion  
2572  USER_0017         NaN        NaN  
150   USER_0222         NaN 

In [32]:
recommendations_word2vec = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations_word2vec.append([user, get_personalized_recommendations(user).articleID.values])

In [33]:
recommendations_word2vec = pd.DataFrame(recommendations_word2vec, columns=['userID', 'articleID'])

In [34]:
recommendations_word2vec

,userID,articleID
0,USER_0000,"[ARTICLE_2572, ARTICLE_0150, ARTICLE_2461, ART..."
1,USER_0001,"[ARTICLE_1104, ARTICLE_2461, ARTICLE_2672, ART..."
2,USER_0002,"[ARTICLE_2672, ARTICLE_0906, ARTICLE_0767, ART..."
3,USER_0003,"[ARTICLE_2672, ARTICLE_0012, ARTICLE_0767, ART..."
4,USER_0004,"[ARTICLE_2672, ARTICLE_0012, ARTICLE_0767, ART..."
...,...,...
1410,USER_1416,"[ARTICLE_0817, ARTICLE_0389, ARTICLE_0843, ART..."
1411,USER_1417,"[ARTICLE_2672, ARTICLE_2196, ARTICLE_0774, ART..."
1412,USER_1418,"[ARTICLE_2672, ARTICLE_0012, ARTICLE_0906, ART..."
1413,USER_1419,"[ARTICLE_0893, ARTICLE_2775, ARTICLE_1608, ART..."


In [35]:
recommendations_word2vec2 = recommendations_word2vec.explode('articleID').reset_index(drop=True)


In [36]:
recommendations_word2vec2

,userID,articleID
0,USER_0000,ARTICLE_2572
1,USER_0000,ARTICLE_0150
2,USER_0000,ARTICLE_2461
3,USER_0000,ARTICLE_1360
4,USER_0000,ARTICLE_0252
...,...,...
7070,USER_1420,ARTICLE_0767
7071,USER_1420,ARTICLE_2196
7072,USER_1420,ARTICLE_0774
7073,USER_1420,ARTICLE_1323


In [ ]:
recommendations_word2vec2.to_csv(data_path + 'submission_w2v_v3.csv')

In [37]:
# topic modeling -> 기사 분류 후 추천

In [38]:
# https://lsjsj92.tistory.com/571

In [93]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# nltk 불용어 다운로드
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 텍스트 전처리 함수
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# 제목과 본문을 결합하여 전처리
article_info['processed'] = (article_info['Title'] + ' ' + article_info['Content']).apply(preprocess)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [96]:
# 벡터화 (Bag of Words)
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(article_info['processed'])

# LDA 모델 학습
lda = LatentDirichletAllocation(n_components=20, random_state=42)  # 10개의 토픽
lda.fit(dtm)


LatentDirichletAllocation(n_components=20, random_state=42)

In [97]:
# 각 문서의 토픽 분포
topic_distributions = lda.transform(dtm)

# 각 문서를 가장 높은 확률의 토픽으로 분류
article_info['Topic'] = topic_distributions.argmax(axis=1)

# 각 토픽의 상위 단어 추출
def get_top_words(model, feature_names, n_top_words):
    top_words = []
    for topic_idx, topic in enumerate(model.components_):
        top_words.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return top_words

n_top_words = 10
feature_names = vectorizer.get_feature_names_out()
top_words = get_top_words(lda, feature_names, n_top_words)

# 출력: 각 토픽의 상위 단어
for i, words in enumerate(top_words):
    print(f"Topic {i}: {', '.join(words)}")

# 예시 사용법: 특정 토픽에 속한 문서들 확인
topic_num = 0
print(article_info[article_info['Topic'] == topic_num][['articleID', 'Title']])


Topic 0: black, like, power, way, time, performance, death, battery, years, plague
Topic 1: drupal, api, service, web, use, load, aws, http, cloud, services
Topic 2: page, google, search, content, site, user, website, time, use, analytics
Topic 3: content, key, data, document, use, filter, tag, used, object, encryption
Topic 4: que, com, para, da, em, uma, testes, como, por, os
Topic 5: people, like, apple, time, work, know, make, things, way, really
Topic 6: que, para, um, em, uma, com, da, os, não, como
Topic 7: em, que, da, para, com, os, um, na, por, dos
Topic 8: jira, dna, portfolio, power, nuclear, kelley, genetic, atlassian, articles, stories
Topic 9: que, não, um, para, uma, da, eu, com, em, na
Topic 10: google, android, app, docker, apps, new, windows, container, run, ios
Topic 11: learning, machine, data, deep, model, image, neural, google, algorithms, training
Topic 12: technology, companies, new, data, company, year, ai, world, says, digital
Topic 13: da, em, para, com, que

In [42]:
article_info

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion,processed,Topic
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN,19 tips everyday git use using git full time p...,3
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN,intel buys computer vision startup itseez impr...,0
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN,practical end end testing protractor one reaso...,2
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN,corporate venture growth brazil another sign m...,0
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN,cross channel user experiences drupal awesome ...,0
...,...,...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG,como consumir conteúdo de qualidade em ios coc...,6
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN,aurelia 1 0 amazing journey get happened witho...,2
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP,lessons converting app 100 kotlin keepsafe eng...,2
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN,ita está oferecendo 10 cursos gratuitos distân...,6


In [105]:
view_article[view_article['Topic'] == 3].shape

(3529, 13)

In [48]:
view_article = pd.merge(view_log, article_info, on = 'articleID')

In [107]:
view_article

,userID_x,articleID,userRegion_x,userCountry_x,Title,Content,Format,Language,userID_y,userCountry_y,userRegion_y,processed,Topic
0,USER_0000,ARTICLE_0661,NY,US,Facebook says its new AI is almost as good as ...,Facebook has developed a new AI called DeepTex...,HTML,en,USER_1304,NaN,NaN,facebook says new ai almost good humans unders...,5
1,USER_0067,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,Facebook has developed a new AI called DeepTex...,HTML,en,USER_1304,NaN,NaN,facebook says new ai almost good humans unders...,5
2,USER_0414,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,Facebook has developed a new AI called DeepTex...,HTML,en,USER_1304,NaN,NaN,facebook says new ai almost good humans unders...,5
3,USER_0557,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,Facebook has developed a new AI called DeepTex...,HTML,en,USER_1304,NaN,NaN,facebook says new ai almost good humans unders...,5
4,USER_1163,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,Facebook has developed a new AI called DeepTex...,HTML,en,USER_1304,NaN,NaN,facebook says new ai almost good humans unders...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42712,USER_1420,ARTICLE_0714,SP,BR,Sponsored: Five reasons why retailers should o...,Unsupported browser For a better experience pl...,HTML,en,USER_1420,BR,SP,sponsored five reasons retailers opt digital r...,1
42713,USER_1420,ARTICLE_1711,SP,BR,Automação de Marketing: o seu negócio está pro...,A automação de marketing está ai para todos? D...,HTML,pt,USER_1420,NaN,NaN,automação de marketing seu negócio está pronto...,6
42714,USER_1420,ARTICLE_1732,SP,BR,Digital Tool as Strategic Enabler for Banking ...,"By Brian Pallas , founder and CEO of Opportuni...",HTML,en,USER_1420,BR,SP,digital tool strategic enabler banking transfo...,0
42715,USER_1420,ARTICLE_1732,SP,BR,Digital Tool as Strategic Enabler for Banking ...,"By Brian Pallas , founder and CEO of Opportuni...",HTML,en,USER_1420,BR,SP,digital tool strategic enabler banking transfo...,0


In [102]:
len(view_article[view_article['Topic'] == 3])

3529

In [113]:
view_article[view_article['Topic'] == 0].articleID.value_counts() # 각 토픽에서 사용자가 많이 본 순서로 추천

articleID
ARTICLE_0561    79
ARTICLE_2099    72
ARTICLE_1745    65
ARTICLE_1901    64
ARTICLE_1884    62
                ..
ARTICLE_1489     1
ARTICLE_0355     1
ARTICLE_0264     1
ARTICLE_2695     1
ARTICLE_0544     1
Name: count, Length: 591, dtype: int64

In [91]:
view_article[view_article['userID_x'] == 'USER_1420'].Topic.value_counts()

Topic
0    27
6    14
5     8
1     4
3     2
7     2
8     2
2     1
Name: count, dtype: int64

In [108]:
article_info[article_info['Topic'] == 0]

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion,processed,Topic
110,ARTICLE_0110,Pinterest's Real-World Pins Let In-Store Shopp...,It's a real pin that looks like a virtual pin ...,HTML,en,USER_0465,NaN,NaN,pinterest real world pins let store shoppers s...,0
119,ARTICLE_0119,How big is the Google Earth database? - Google...,This is the fourth and final in a series of po...,HTML,en,USER_0566,NaN,NaN,big google earth database google earth blog fo...,0
127,ARTICLE_0127,This Battery-Free Computer Sucks Power Out Of ...,"Today, the biggest hurdle when it comes to des...",HTML,en,USER_1304,NaN,NaN,battery free computer sucks power thin air tod...,0
138,ARTICLE_0138,Mining in Space Could Lead to Conflicts on Ear...,Illustration by Maciej Frolow Space mining is ...,HTML,en,USER_0566,NaN,NaN,mining space could lead conflicts earth facts ...,0
301,ARTICLE_0301,"Why 3,000 bank branches must close","I'm the Money editor of the Guardian, but have...",HTML,en,USER_1242,NaN,NaN,3 000 bank branches must close money editor gu...,0
318,ARTICLE_0318,Whirlpool Vessi puts a microbrewery in your ki...,If you were waiting for proof that home brewin...,HTML,en,USER_0827,NaN,NaN,whirlpool vessi puts microbrewery kitchen news...,0
425,ARTICLE_0425,USB-C and Lightning headphones,There are literally billions of headphones out...,HTML,en,USER_0566,NaN,NaN,usb c lightning headphones literally billions ...,0
478,ARTICLE_0478,The Black Death: The Greatest Catastrophe Ever,he disastrous mortal disease known as the Blac...,HTML,en,USER_0566,NaN,NaN,black death greatest catastrophe ever disastro...,0
756,ARTICLE_0756,Request lesson : How and when to use はず(=hazu)...,= Kotoshi no aki made niwa kare ga dekiru hazu...,HTML,la,USER_0617,NaN,NaN,request lesson use はず hazu maggie sensei kotos...,0
939,ARTICLE_0939,Sony Smart Tennis Sensor for Tennis Rackets,Check worldwide shot data and improve your gam...,HTML,en,USER_1213,NaN,NaN,sony smart tennis sensor tennis rackets check ...,0


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
# 사용자별로 읽은 기사들의 카테고리 정보 추출
user_categories = view_log.merge(article_info[['articleID', 'Topic']], on='articleID')

# 각 사용자별로 가장 많이 읽은 카테고리 추출
user_top_categories = user_categories.groupby('userID')['Topic'].agg(lambda x: x.value_counts().idxmax()).reset_index()
user_top_categories.columns = ['userID', 'TopCategory']


In [112]:
# 개인화된 추천 시스템 함수
def get_category_based_recommendations(user_id, user_top_categories, article_info, view_log):
    # 사용자의 관심 카테고리 가져오기
    user_category = user_top_categories[user_top_categories['userID'] == user_id]['TopCategory'].values
    if len(user_category) == 0:
        return pd.DataFrame()  # 사용자의 관심 카테고리가 없으면 빈 데이터프레임 반환
    user_category = user_category[0]

    # 사용자가 읽은 기사 제외
    user_read_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 관심 카테고리의 기사 중에서 추천할 기사 선택
    recommended_articles = view_article[view_article['Topic'] == 0].articleID.value_counts().index # 각 토픽에서 사용자가 많이 본 순서로 추천

    # 상위 5개 기사 추천 (카테고리 내에서 임의로 선택)
    return recommended_articles[:5]

# 예시 사용법
user_id = 'USER_1420'
recommended_articles = get_category_based_recommendations(user_id, user_top_categories, article_info, view_log)
print(recommended_articles)


Index(['ARTICLE_0561', 'ARTICLE_2099', 'ARTICLE_1745', 'ARTICLE_1901',
       'ARTICLE_1884'],
      dtype='object', name='articleID')


In [115]:
category_based_recommendations = []
for idx, user in enumerate(view_log.userID.unique()):
    category_based_recommendations.append([user, get_category_based_recommendations(user, user_top_categories, article_info, view_log)])

In [116]:
category_based_recommendations = pd.DataFrame(category_based_recommendations, columns=['userID', 'articleID'])
category_based_recommendations = category_based_recommendations.explode('articleID').reset_index(drop=True)

In [117]:
category_based_recommendations

,userID,articleID
0,USER_0000,ARTICLE_0561
1,USER_0000,ARTICLE_2099
2,USER_0000,ARTICLE_1745
3,USER_0000,ARTICLE_1901
4,USER_0000,ARTICLE_1884
...,...,...
7070,USER_1420,ARTICLE_0561
7071,USER_1420,ARTICLE_2099
7072,USER_1420,ARTICLE_1745
7073,USER_1420,ARTICLE_1901


In [118]:
category_based_recommendations.to_csv(data_path + 'category_based_recommendations_v2.csv')

In [86]:
def get_diverse_recommendations(user_id, user_top_categories, article_info, view_log, top_n=5, diversity_ratio=0.5):
    # 사용자의 관심 카테고리 가져오기
    user_category = user_top_categories[user_top_categories['userID'] == user_id]['TopCategory'].values
    if len(user_category) == 0:
        return pd.DataFrame()  # 사용자의 관심 카테고리가 없으면 빈 데이터프레임 반환
    user_category = user_category[0]

    # 사용자가 읽은 기사 제외
    user_read_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 관심 카테고리의 기사 추천
    category_articles = article_info[(article_info['Topic'] == user_category) & (~article_info['articleID'].isin(user_read_articles))]
    num_category_recommendations = int(top_n * diversity_ratio)
    category_recommendations = category_articles.head(num_category_recommendations)

    # 나머지 다양한 카테고리의 기사 추천
    other_articles = article_info[(article_info['Topic'] != user_category) & (~article_info['articleID'].isin(user_read_articles))]
    num_other_recommendations = top_n - num_category_recommendations
    other_recommendations = other_articles.sample(num_other_recommendations)

    # 추천 기사 결합
    recommendations = pd.concat([category_recommendations, other_recommendations])

    return recommendations

# 예시 사용법
user_id = 'USER_1245'
recommended_articles = get_diverse_recommendations(user_id, user_top_categories, article_info, view_log)
print(recommended_articles)


         articleID                                              Title  \
5     ARTICLE_0005  Conheça o Relatório de Informações do Público ...   
18    ARTICLE_0018  Microsoft comprará LinkedIn por 26,2 bilhões d...   
1566  ARTICLE_1566  A Googler analyzed a billion files to settle t...   
1361  ARTICLE_1361  O Brasil das fintechs: conheça startups nacion...   
2868  ARTICLE_2868  Former Google career coach shares a visual tri...   

                                                Content Format Language  \
5     Já havíamos falado no blog sobre como conhecer...   HTML       pt   
18    São Paulo - A Microsoft anunciou hoje que comp...   HTML       pt   
1566  If you watch HBO's "Silicon Valley," you may r...   HTML       en   
1361  SÃO PAULO - Já faz algum tempo que as startups...   HTML       pt   
2868  If you want 2017 to be an exciting year, desig...   HTML       en   

         userID userCountry userRegion  \
5     USER_1324         NaN        NaN   
18    USER_0063         Na

In [83]:
category_based_recommendations = []
for idx, user in enumerate(view_log.userID.unique()):
    category_based_recommendations.append([user, get_diverse_recommendations(user, user_top_categories, article_info, view_log).articleID.values])

In [84]:
category_based_recommendations = pd.DataFrame(category_based_recommendations, columns=['userID', 'articleID'])
category_based_recommendations = category_based_recommendations.explode('articleID').reset_index(drop=True)

In [85]:
category_based_recommendations

,userID,articleID
0,USER_0000,ARTICLE_0001
1,USER_0000,ARTICLE_0003
2,USER_0000,ARTICLE_0594
3,USER_0000,ARTICLE_0426
4,USER_0000,ARTICLE_0762
...,...,...
7067,USER_1420,ARTICLE_0001
7068,USER_1420,ARTICLE_0003
7069,USER_1420,ARTICLE_0963
7070,USER_1420,ARTICLE_1872


In [87]:
category_based_recommendations.to_csv(data_path + 'category_based_recommendations.csv')